In [1]:
import numpy as np
import time,subprocess,datetime,sys,glob,random
from astropy.io import fits
sys.path.append('/home/ccd/ucd-scripts/python-lib')
import Email_Warning

date=time.strftime("%Y%m%d")
imagedir='/mnt/10TBHDD/data/'+date

In [3]:
def take_gain_flats(cfgfile):
    try:
        subprocess.run('ccs-script /home/ccd/ucd-scripts/ucd-data.py '+cfgfile,check=True, shell=True)
    except:
        power_light("off")
        raise Exception("Error in taking gain flats. Check on CCD state!")
    return

def power_light(state):
    if state=="off" or state=="OFF" or state=="Off":
        subprocess.run('ccs-script /home/ccd/ucd-scripts/scripts/sphereOff.py',check=True, shell=True)
    elif state=="on" or state=="ON" or state=="On":
        subprocess.run('ccs-script /home/ccd/ucd-scripts/scripts/sphereOn.py',check=True, shell=True)

In [5]:
def calculate_gains():
    fitsfiles=np.array(glob.glob(imagedir+'/*.fits'))
    filetypes=np.array([fits.getheader(name)["IMGTYPE"] for name in fitsfiles])
    flats=fitsfiles[np.where(filetypes=="FLAT")]
    biases=fitsfiles[np.where(filetypes=="BIAS")]
    if len(flats)!=2:
        raise Exception("wrong number of fits images for gain estimation")
    fitsfiles.sort()

    biasmedian=np.median(np.array([fits.getdata(bias,3) for bias in biases]))
    lowmedian=np.median(fits.getdata(flats[0],3))
    highmedian=np.median(fits.getdata(flats[1],3))
    lowdiff=lowmedian-biasmedian
    highdiff=highmedian-biasmedian

    fivethousandADUintensitylow=(5000/lowdiff)
    fivethousandADUintensityhigh=(5000/highdiff)*5
    fivethousandADUintensity=(fivethousandADUintensitylow+fivethousandADUintensityhigh)/2

    levels='''Bias level: '''+str(biasmedian)+'''
    Low flat level: '''+str(lowmedian)+'''
    High flat level: '''+str(highmedian)+'''

    5,000ADU intensity: '''+str(fivethousandADUintensity)+'''
    50,000 intensity: '''+str(fiftythousandADUintensity)
    file = open(imagedir+'/gaininfo.txt', 'w')
    file.write(levels)       
    file.close()
    
    return fivethousandADUintensity
    

In [9]:
fivethousandADU='1  6'
fiftythousandADU='1  6'

In [2]:
def find_ptc_values(resolution,fivethousandADUintensity):
    topint=fivethousandADUintensity*5
    minlamp=0.3
    minexp=1.0
    maxexp=15.0
    maxlamp=67.9
    bad1=6.4
    bad2=12
    ints=np.linspace(0.3,topint,resolution)
    
    #flat pairs
    exp=minexp
    explist=[]

    for i in ints:
        if minlamp<=i/exp<=bad1 or bad2<=i/exp<=maxlamp:
            explist.append(str(exp)+"   "+str("{:.2f}".format(i/exp))+"  2,")
        elif bad1<=i/exp<=bad2:
            explist.append(str(exp*2)+"   "+str("{:.2f}".format(i/(exp*2)))+"  2,")
        elif i/exp>=maxlamp:
            while i/exp>=maxlamp:
                exp+=0.5
            explist.append(str(exp)+"   "+str("{:.2f}".format(i/exp))+"  2,")
    random.shuffle(explist)
    explist[-1]=explist[-1][:-1]
    ptcintensities=explist[0]
    for i in explist[1:]:
        ptcintensities=ptcintensities+'''
    '''+i
    return ptcintensities

SyntaxError: invalid syntax (464492264.py, line 2)

In [11]:
#flat pairs
exp=minexp
explist=[]

for i in ints:
    if minlamp<=i/exp<=bad1 or bad2<=i/exp<=maxlamp:
        explist.append(str(exp)+"   "+str("{:.2f}".format(i/exp))+"  2,")
    elif bad1<=i/exp<=bad2:
        explist.append(str(exp*2)+"   "+str("{:.2f}".format(i/(exp*2)))+"  2,")
    elif i/exp>=maxlamp:
        while i/exp>=maxlamp:
            exp+=0.5
        explist.append(str(exp)+"   "+str("{:.2f}".format(i/exp))+"  2,")
random.shuffle(explist)
explist[-1]=explist[-1][:-1]
ptcintensities=explist[0]
for i in explist[1:]:
    ptcintensities=ptcintensities+'''
'''+i
print(ptcintensities)

1.0   27.92  2,
1.5   57.49  2,
1.0   55.55  2,
1.0   43.27  2,
1.0   58.62  2,
1.5   59.54  2,
1.0   12.58  2,
1.0   15.65  2,
1.0   64.75  2,
2.0   58.47  2,
1.0   0.30  2,
2.0   4.75  2,
2.0   3.22  2,
1.0   67.82  2,
1.0   49.41  2,
1.5   63.63  2,
1.0   40.20  2,
1.5   67.72  2,
2.0   55.40  2,
1.0   34.06  2,
2.0   60.00  2,
2.0   52.33  2,
2.0   53.86  2,
1.0   61.68  2,
1.5   47.26  2,
1.0   30.99  2,
1.0   52.48  2,
1.5   61.58  2,
1.0   37.13  2,
1.5   55.45  2,
1.0   46.34  2,
1.5   49.31  2,
1.5   65.68  2,
1.5   51.35  2,
1.0   24.85  2,
1.0   21.78  2,
2.0   56.93  2,
1.0   18.72  2,
1.0   3.37  2,
1.5   53.40  2


In [8]:
def find_persistence(fivethousandADUintensity,ADUlevel):
    persistenceintensities=fivethousandADUintensity*ADUlevel/5000
    persistenceexposure=1
    while persistenceintensities>=maxlamp:
        persistenceexposure*=2
        persistenceintensities/=2
    persistence=str(persistenceexposure)+"  "+str(persistenceintensities)
    return persistence

In [9]:
def make_cfg(filename,fivethousandADUintensity,lowsuperflatADUlevel,highsuperflatADUlevel,ptc,persistence):
    cfgfile='''#UCD Sequencer test file
# 
# Acquisition sequences to run
[ACQUIRE]
bias1
bias2
bias3
bias4
dark
superflat
ptc
persistenceflats
persistencedarks

[DESCRIPTION]
A full run for a given sequencer file.

[BIAS1]
ACQTYPE=bias
LOCATIONS = R21/Reb0        # locations to read
COUNT     = 5              # number of bias frames
EXTRADELAY = 0

[BIAS2]
ACQTYPE=bias
LOCATIONS = R21/Reb0        # locations to read
COUNT     = 5              # number of bias frames
EXTRADELAY = 15

[BIAS3]
ACQTYPE=bias
LOCATIONS = R21/Reb0        # locations to read
COUNT     = 5              # number of bias frames
EXTRADELAY = 30

[BIAS4]
ACQTYPE=bias
LOCATIONS = R21/Reb0        # locations to read
COUNT     = 5              # number of bias frames
EXTRADELAY = 60

[DARK]
LOCATIONS=R21/Reb0
DESCRIPTION=Darks.
BCOUNT=    1      # number of bias frames per dark image
dark= 30  5,      # integration time and image count for dark set
      180 5,
      360 5

[SUPERFLAT]
ACQTYPE=flat
DESCRIPTION  = Superflat with 20 flats at 5000 and 50000 ADU.
LOCATIONS    = R21/Reb0     # Locations to read
BCOUNT       = 1            # number of bias frames per flat set
WL           = r            # wavelength filter to use for the flats

flat =  1  '''+str(fivethousandADUintensity*lowsuperflatADULevel/5000)+'''   25,    
        1  '''+str(fivethousandADUintensity*highsuperflatADULevel/5000)+'''   25
        
        
[PTC]
ACQTYPE=flat
DESCRIPTION  = Superflat with 20 flats at 5000 and 50000 ADU.
LOCATIONS    = R21/Reb0     # Locations to read
BCOUNT       = 1            # number of bias frames per flat set
WL           = r            # wavelength filter to use for the flats
flat = '''+ptc+'''

[PERSISTENCEFLATS]
ACQTYPE=flat
DESCRIPTION  = Superflat with 20 flats at 5000 and 50000 ADU.
LOCATIONS    = R21/Reb0     # Locations to read
BCOUNT       = 21            # number of bias frames per flat set
WL           = r            # wavelength filter to use for the flats
flat = '''+persistence+'''  1

[PERSISTENCEDARKS]
ACQTYPE=dark
LOCATIONS=R21/Reb0
DESCRIPTION=Darks.
BCOUNT=    0      # number of bias frames per dark image
dark= 15   40'''
    file = open(filename, 'w')
    file.write(cfgfile)       
    file.close()
    return

In [15]:
file = open('runninglog.cfg', 'w')
file.write(cfgfile)       
file.close()

In [10]:
def remove_fits_files(imagedir):
    subprocess.run('rm '+imagedir+'/TS_C_*',check=True, shell=True)

In [11]:
def copy_file_to_imagedir(file):
    subprocess.run('cp '+file+" "+imagedir,check=True, shell=True)

In [ ]:
def full_gain_calculation():
    take_gain_flats(gaincfgfile)
    fivethousandlevel=calculate_gains(imagedir)
    ptc=find_ptc_values(ptcresolution,fivethousandlevel)
    persistence=find_persistence(fivethousandADUintensity,persistenceADUlevel)
    cfgwords=make_cfg(sequencercfgfile,fivethousandlevel,lowsuperflatADUlevel,highsuperflatADUlevel,ptc,persistence)
    remove_fits_files(imagedir)
    copy_file_to_imagedir(sequencercfgfile)

In [12]:
def move_files_to_new_directory(imagedir,sequencerfilename):
    subprocess.run('mkdir '+imagedir+"/"+sequencerfilename,check=True, shell=True)
    subprocess.run('mv '+imagedir+"/TS_C* "+imagedir+"/"+sequencerfilename,check=True, shell=True)

In [ ]:
def full_sequencer_run(sleeptime):
    time.sleep(sleeptime)
    eWarning("Starting new sequencer run.")
    take_data(sequencercfgfile)
    copy_file_to_imagedir(sequencercfgfile)
    move_files_to_new_directory(imagedir,sequencerfilename)
    eWarning("Finished sequencer run.")

In [3]:
def get_sequencer_from_header():
    fitsfiles=np.array(glob.glob(imagedir+'/*.fits'))
    headerseq=fits.getheader(fitsfiles[0])["SEQFILE"]
    return headerseq
    

In [4]:
seq=get_sequencer_from_header()

In [7]:
seq!="FP_E2V_2s_ir2_v27_rt450"+".seq"

False

In [2]:
def power_CCD(state):
    if state=="off" or state=="OFF" or state=="Off":
        subprocess.run('ccs-script /home/ccd/ucd-scripts/scripts/setBackBias.py R21 --off',check=True, shell=True)
        time.sleep(2)
        subprocess.run('ccs-script /home/ccd/ucd-scripts/scripts/powerCCD.py R21 --off',check=True, shell=True)
    elif state=="on" or state=="ON" or state=="On":
        subprocess.run('ccs-script /home/ccd/ucd-scripts/scripts/powerCCD.py R21 --on',check=True, shell=True)
        time.sleep(2)
        subprocess.run('ccs-script /home/ccd/ucd-scripts/scripts/setBackBias.py R21 --on',check=True, shell=True)

In [3]:
power_CCD("on")

[2023-12-10T15:06:29.671PST] WARNING: Unable to read default leap second file, falling back to reading local copy from bootstrap path (org.lsst.ccs.utilities.taitime.CCSTimeStamp <clinit>)
[2023-12-10T15:06:29.688PST] SEVERE: Leap second data read from file:/home/ccd/ccs/org-lsst-ccs-ucd-software-main-1.0.0-SNAPSHOT/etc/leap-seconds.list has expired (org.lsst.ccs.utilities.taitime.CCSTimeStamp <clinit>)


True


[2023-12-10T15:06:38.265PST] WARNING: Unable to read default leap second file, falling back to reading local copy from bootstrap path (org.lsst.ccs.utilities.taitime.CCSTimeStamp <clinit>)
[2023-12-10T15:06:38.276PST] SEVERE: Leap second data read from file:/home/ccd/ccs/org-lsst-ccs-ucd-software-main-1.0.0-SNAPSHOT/etc/leap-seconds.list has expired (org.lsst.ccs.utilities.taitime.CCSTimeStamp <clinit>)


BSS supply on.
True
Setting back bias switch on.
True


In [4]:
power_CCD("off")

[2023-12-10T15:06:58.788PST] WARNING: Unable to read default leap second file, falling back to reading local copy from bootstrap path (org.lsst.ccs.utilities.taitime.CCSTimeStamp <clinit>)
[2023-12-10T15:06:58.802PST] SEVERE: Leap second data read from file:/home/ccd/ccs/org-lsst-ccs-ucd-software-main-1.0.0-SNAPSHOT/etc/leap-seconds.list has expired (org.lsst.ccs.utilities.taitime.CCSTimeStamp <clinit>)


Setting back bias switch off.
BSS supply off.
True
True


[2023-12-10T15:07:05.646PST] WARNING: Unable to read default leap second file, falling back to reading local copy from bootstrap path (org.lsst.ccs.utilities.taitime.CCSTimeStamp <clinit>)
[2023-12-10T15:07:05.666PST] SEVERE: Leap second data read from file:/home/ccd/ccs/org-lsst-ccs-ucd-software-main-1.0.0-SNAPSHOT/etc/leap-seconds.list has expired (org.lsst.ccs.utilities.taitime.CCSTimeStamp <clinit>)


Waiting 5.0 seconds.
CCD power off.
True


In [128]:
def change_sequencer(sequencer):
    text='''set target ucd-fp

sequencerConfig change sequencer [E2V:FP_E2V_2s_ir2_'''+sequencer+'''.seq,ITL:FP_ITL_2s_ir2_'''+sequencer+'''.seq]

printConfigurationParameters Sequencer'''
    file = open("ccssequencercommands.txt", 'w')
    file.write(text)       
    file.close()
    
    output=subprocess.check_output(["/home/ccd/ccs/bin/ccs-shell <ccssequencercommands.txt"],shell=True)
    return output

In [90]:
def check_sequencer():
    output=subprocess.check_output(["/home/ccd/ccs/bin/ccs-shell <ccssequencercheck.txt"],shell=True)
    output=str(output)
    output=output.split("\\n")
    output=output[-11]
    output=output.split("ir2_")
    ITLseqreturn=output[2][:-6]
    e2vseqreturn=output[1].split(".seq")
    e2vseqreturn=e2vseqreturn[0]
    return e2vseqreturn,ITLseqreturn

In [107]:
a=check_sequencer()

[2023-12-10T17:17:09.820PST] WARNING: Unable to read default leap second file, falling back to reading local copy from bootstrap path (org.lsst.ccs.utilities.taitime.CCSTimeStamp <clinit>)
[2023-12-10T17:17:09.834PST] SEVERE: Leap second data read from file:/home/ccd/ccs/org-lsst-ccs-ucd-software-main-1.0.0-SNAPSHOT/etc/leap-seconds.list has expired (org.lsst.ccs.utilities.taitime.CCSTimeStamp <clinit>)


In [108]:
a

('v26', 'v26')

In [56]:
b=b.split("\\n")

In [57]:
b

["b'Type help for list of available commands",
 ' ccs>set target ucd-fp',
 'ucd-fp ccs>',
 'ucd-fp ccs>printConfigurationParameters Sequencer',
 'Category : Sequencer',
 'sequencerConfig clearCountParameter ClearCount unitless',
 'sequencerConfig clearMain Clear unitless',
 'sequencerConfig idleFlushMain Idle unitless',
 'sequencerConfig idleFlushTimeout -1 unitless',
 'sequencerConfig integrateMain Integrate unitless',
 'sequencerConfig metaDataRegisters [UnderCols,PreCols,ReadCols,PostCols,ReadCols2,OverCols,PreRows,ReadRows,PostRows,OverRows,OpFlags] ',
 'sequencerConfig overCols -1 ',
 'sequencerConfig overRows -1 ',
 'sequencerConfig postCols -1 ',
 'sequencerConfig postRows -1 ',
 'sequencerConfig preCols -1 ',
 'sequencerConfig preRows -1 ',
 'sequencerConfig pseudoReadMain PseudoRead unitless',
 'sequencerConfig readCols -1 ',
 'sequencerConfig readCols2 -1 ',
 'sequencerConfig readMain Read unitless',
 'sequencerConfig readRows -1 ',
 'sequencerConfig rowShiftForwardMain RowSh

In [66]:
c=b[-11]

In [68]:
d=c.split("ir2_")

In [72]:
d[2][:-6]

'v26'

In [75]:
e=d[1].split(".seq")

In [76]:
e

['v26', ',ITL:FP_ITL_2s_']

In [129]:
a=change_sequencer("v27_rt450")

[2023-12-10T17:24:34.957PST] WARNING: Unable to read default leap second file, falling back to reading local copy from bootstrap path (org.lsst.ccs.utilities.taitime.CCSTimeStamp <clinit>)
[2023-12-10T17:24:34.971PST] SEVERE: Leap second data read from file:/home/ccd/ccs/org-lsst-ccs-ucd-software-main-1.0.0-SNAPSHOT/etc/leap-seconds.list has expired (org.lsst.ccs.utilities.taitime.CCSTimeStamp <clinit>)


In [131]:
a=check_sequencer()
'v27_rt450'==a[0]==a[1]

[2023-12-10T17:26:21.812PST] WARNING: Unable to read default leap second file, falling back to reading local copy from bootstrap path (org.lsst.ccs.utilities.taitime.CCSTimeStamp <clinit>)
[2023-12-10T17:26:21.825PST] SEVERE: Leap second data read from file:/home/ccd/ccs/org-lsst-ccs-ucd-software-main-1.0.0-SNAPSHOT/etc/leap-seconds.list has expired (org.lsst.ccs.utilities.taitime.CCSTimeStamp <clinit>)


True

In [89]:
e2vseqreturn

'v27_rt450'

In [132]:
not 1==1==1

False

In [139]:
sequencerlist=["v26","v27_rt150","v27_rt300","v27_rt450","v27_rt600","v27_rt750","v27_rt750_iso1_140","v27_rt750_iso1_210","v27_rt750_iso1_70","v27_rt750_iso2_150","v27_rt750_iso2_300","v27_rt750_iso2_450"]

In [135]:
def full_sequencer_run(sleeptime):
    print("Get outta there! Sleeping for "+str(sleeptime)+"s")
    time.sleep(sleeptime)
    eWarning("Starting new sequencer run.")
    i=1
    length=str(len(sequencerlist))
    for sequencerfilename in sequencerlist:
        change_sequencer(sequencerfilename)
        time.sleep(5)
        check=check_sequencer()
        if not check[0]==check[1]==sequencerfilename:
            eWarning("The sequencer updated incorrectly for "+str(sequencerfilename))
            time.sleep(2)
            raise Exception("The sequencer updated incorrectly for "+str(sequencerfilename))
        try:
            power_CCD("on")
            take_data(sequencercfgfile)
            sequencer="FP_E2V_2s_ir2_"+sequencerfilename+".seq"
            seq=get_sequencer_from_header()
            if sequencer!=seq:
                eWarning("The sequencer file is not correct in the header for "+str(sequencerfilename))
                time.sleep(2)
                raise Exception("Sequencer file not correct in the header!")
            copy_file_to_imagedir(sequencercfgfile)
            move_files_to_new_directory(imagedir,sequencerfilename)
            power_CCD("off")
            eWarning("Finished sequencer run for "++str(sequencerfilename)+" "+str(i)+"/"+length)
            i+=1
        except:
            eWarning("Error in sequencer run on "+str(sequencerfilename))
            raise Exception("Error in sequencer run on "+str(sequencerfilename))

In [140]:
for i in sequencerlist:
    print(i)

v26
v27_rt150
v27_rt300
v27_rt450
v27_rt600
v27_rt750
v27_rt750_iso1_140
v27_rt750_iso1_210
v27_rt750_iso1_70
v27_rt750_iso2_150
v27_rt750_iso2_300
v27_rt750_iso2_450


In [9]:
topint=20*5
minlamp=15
minexp=1.0
maxexp=15.0
maxlamp=67.9
bad1=6.4
bad2=12
ints=np.linspace(0.3,topint,40)

#flat pairs
exp=minexp
explist=[]

for i in ints:
    if minlamp<=i/exp<=bad1 or bad2<=i/exp<=maxlamp:
        explist.append(str(exp)+"   "+str("{:.2f}".format(i/exp))+",")
    elif bad1<=i/exp<=bad2:
        explist.append(str(exp*2)+"   "+str("{:.2f}".format(i/(exp*2)))+",")
    elif i/exp>=maxlamp:
        while i/exp>=maxlamp:
            exp+=0.5
        explist.append(str(exp)+"   "+str("{:.2f}".format(i/exp))+",")
random.shuffle(explist)
explist[-1]=explist[-1][:-1]
ptcintensities=explist[0]
for i in explist[1:]:
    ptcintensities=ptcintensities+'''
'''+i

In [10]:
ptcintensities

'1.5   46.22,\n1.0   61.65,\n1.0   46.32,\n1.0   18.19,\n1.0   20.75,\n1.0   30.98,\n1.5   53.03,\n1.0   43.76,\n1.5   59.85,\n1.0   51.43,\n1.5   64.96,\n1.0   23.31,\n1.0   59.10,\n1.0   56.54,\n1.0   66.77,\n1.5   49.62,\n1.5   61.55,\n1.5   66.67,\n1.0   25.86,\n1.0   53.98,\n2.0   5.26,\n1.5   58.15,\n1.5   63.26,\n1.5   56.44,\n2.0   3.98,\n1.0   15.64,\n1.5   54.74,\n1.0   38.65,\n1.0   48.87,\n1.0   33.53,\n1.0   13.08,\n1.5   51.33,\n1.0   28.42,\n1.0   41.20,\n1.5   47.92,\n1.0   64.21,\n1.0   36.09'

In [11]:
file = open("tester.txt", 'w')
file.write(ptcintensities)       
file.close()